BERT as feature extractor.Classifier Naive Bayes

In [2]:
!pip install transformers pandas scikit-learn nltk


  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 132.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
import torch


In [4]:
data = pd.read_csv('/content/Restaurant_Reviews.csv')
data.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
X = data['Review']
y = data['Liked']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def tokenize_text(text):
    tokens = tokenizer.encode_plus(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return tokens['input_ids'], tokens['attention_mask']

X_train_tokens = [tokenize_text(text) for text in X_train]
X_test_tokens = [tokenize_text(text) for text in X_test]


In [24]:


def extract_embeddings(tokens):
    with torch.no_grad():
        model_output = model(tokens[0], attention_mask=tokens[1])
    return model_output.pooler_output

In [ ]:
from sklearn.linear_model import LogisticRegression

Checked the classification using MNB.but couldnt proceed since it couldnt accept negative values.But bert produces positive and negative values

In [26]:
X_train_embeddings = torch.cat([extract_embeddings(tokens) for tokens in X_train_tokens])
X_test_embeddings = torch.cat([extract_embeddings(tokens) for tokens in X_test_tokens])

# Train the Naive Bayes classifier
classifier = LogisticRegression()
classifier.fit(X_train_embeddings, y_train)

# Predict using the trained classifier
y_pred = classifier.predict(X_test_embeddings)

# Evaluate the classifier
print(classification_report(y_test, y_pred))

NameError: ignored